In [ ]:
#### Load the libraries 

import os
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", None, "display.max_columns", None)

## Oxford Economics data

File downloaded from Oxford Economics on 17th of June 2021
Last data Update date is 30th of April 2021

#### Load the file with GVA and Employment

In [ ]:
df = pd.read_excel('F:\\Research\\CommercialPrivate\\Central London\\DATA\\FORECASTS\\2021\\Q2 2021\\Greater London GVA and Employment - 17 June 2021 10_48_24.xlsx', 'Default')

#### Look at the data

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
list(df)

#### Reshape data

In [ ]:
## Reshape 1: Turn years columns into one

df = pd.melt(df, id_vars=('Location',
 'Indicator',
 'Sector',
 'Units',
 'Scale',
 'Measurement',
 'Source',
 'Seasonally adjusted',
 'Base year price',
 'Base year index',
 'Historical end',
 'Date of last update',
 'Source details',
 'Additional source details',
 'Contact email',
 'Location code',
 'Indicator code')
  , var_name="year", value_name="value" )

In [ ]:
#df

In [ ]:
## Reshape 2: Make sectors as columns
## a lot of columns will have to go so that reshape is efficient
# reset index gets rid of the multilevel structure

df = pd.pivot_table(df, values='value', index=['Location',
 'Indicator',
 'Units',
 'Scale',
 'Measurement',
 'year'],
columns=['Sector']).reset_index()



In [ ]:
#df

In [ ]:
list(df)

In [ ]:
df["Aggr2"] = df['Real estate activities'] + df['Financial and insurance'] +  df['Professional, scientific and tech'] + df['Information and communication'] + df['Administrative and support']

In [ ]:
df["Aggr3"] = df["Aggr2"] + df['Public administration and defence']  + df['Education']   

In [ ]:
df["Aggr1"] = df["Aggr2"] - df['Information and communication'] 

In [ ]:
list(df)

In [ ]:
df_aggr = df.iloc[:, [0,1,2,3,4,5, 10, 11, 15, 16, 17]]

In [ ]:
#df_aggr

In [ ]:
# looks like 'Office aggregate (incl. ICT)' is the same as Aggr2 
# and 'Office aggregate (excl. ICT)' is the same as Aggr1
# re-create df_aggr w/t these columns
df_aggr = df.iloc[:, [0,1,2,3,4,5, 15, 16, 17]]

In [ ]:
#df_aggr
## Aggr 1 - excluding ICT
## Aggr 2 - including ICT
## Aggr 3 - including Public admistration and defence & Education

In [ ]:
list(df_aggr)

In [ ]:
## Reshape 3: Turn Aggr columns into one

df_aggr = pd.melt(df_aggr, id_vars=('Location',
 'Indicator',
 'Units',
 'Scale',
 'Measurement',
 'year')
  , var_name="Aggr_version", value_name="value" )

In [ ]:
# create future column names

df_aggr["Aggr_version"]  = df_aggr['Aggr_version'].str[-1:]
df_aggr["Indic_group"] = np.where(df_aggr['Indicator'] == "GVA by Sector", "GVA"+ df_aggr["Aggr_version"], "Employment"+ df_aggr["Aggr_version"]) 


In [ ]:
# Final Reshape
## Reshape 4: Turn Aggregate Indicators into columns

df_aggr = df_aggr.pivot_table( values='value', 
index=['year'],
columns=['Indic_group']).reset_index()


In [ ]:
df_aggr['year'] = df_aggr['year'].astype(int)

In [ ]:
#df_aggr

#### Load the GDP Deflator file

In [ ]:
defl = pd.read_excel('F:\\Research\\CommercialPrivate\\Central London\\DATA\\FORECASTS\\2021\\Q2 2021\\GDP deflator - 21 June 2021 11_11_10.xlsx', 'Default')

In [ ]:
# only keep relevant columns and rows
defl = defl[['Year','Level values']]
defl = defl[defl['Level values'] != "Level values"]

In [ ]:
# rename columns
defl=defl.rename(columns = {'Level values':'GDP deflator', 'Year':'year'})

In [ ]:
#defl

## PMA data

File obtained from PMA on 7th of June 2021. The data we are using is Prime rent and Stock series

#### Load the data on Prime Rents and Stock

In [ ]:
rent = pd.read_excel('F:\\Research\\CommercialPrivate\\Central London\\DATA\\FORECASTS\\2021\\Q2 2021\\Central London fundamentals Jun 2021 - PMA.xls', 'PRIME RENTS - GBP PSF PA', skiprows = 9)

In [ ]:
stock = pd.read_excel('F:\\Research\\CommercialPrivate\\Central London\\DATA\\FORECASTS\\2021\\Q2 2021\\Central London fundamentals Jun 2021 - PMA.xls', 'STOCK', skiprows = 9)

#### Look at the data

In [ ]:
stock.head()

In [ ]:
pma = rent.append(stock, ignore_index=True)

In [ ]:
list(pma)

#### Reshape data

In [ ]:
# Reshape 1: Turn years into 1 column

pma = pd.melt(pma, id_vars=('Market: Submarket','Country', 'Description',)
  , var_name="year", value_name="value" )

In [ ]:
# Remove London from the submarket name
pma['Market: Submarket'] = pma['Market: Submarket'].str.split(': ').str[1]

# rename Submarket column
pma=pma.rename(columns = {'Market: Submarket':'Submarket'})

In [ ]:
# Reshape 2: Turn Prime rent and Stock into columns

pma = pma.pivot_table(values='value', index=['year', 'Submarket'], columns=['Description']).reset_index()

In [ ]:
# rename Submarket column
pma=pma.rename(columns = {'Prime Rents (GBP, psf, pa)':'prime_rent', 'Stock (000s, sqft)':'stock' })

In [ ]:
pma = pma.sort_values(by=['Submarket', 'year'])

## KF data

KF data we use is developemnt pipeline data - to forecast the stock

#### Load the data on Dev Pipeline

In [ ]:
dev_pipe = pd.read_excel('F:\\Research\\CommercialPrivate\\Central London\\DATA\\FORECASTS\\2021\\Q2 2021\\London Development Pipeline Review and Viability Index - Landsec Additions.xlsx', 'London Dev. Pipeline', skiprows = [0,2,3])

In [ ]:
#dev_pipe

#### Slicing and aggregating the data

Slicing and aggregating the data to get the total pipeline values for City and West End submarkets

In [ ]:
# drop the first empty column
dev_pipe = dev_pipe.drop('Unnamed: 0', 1)

In [ ]:
list(dev_pipe)

In [ ]:
# filter data - only keep records with high certainty (Overall 9 and 10)
# only keep records that will complete from 2021 onwards
dev_pipe = dev_pipe[(dev_pipe["Overall"]==9) | (dev_pipe["Overall"]==10)]
dev_pipe = dev_pipe[dev_pipe["Earliest Possible PC Year"]>2020]

In [ ]:
# Select the records that are City in PMA definition - EC1, EC2, EC3, EC4 postcodes
dev_pipe["New_subm"] = np.where(dev_pipe['Address'].str.contains("EC1|EC2|EC3|EC4"),"City",dev_pipe["Submarket"]) 

In [ ]:
# creating West End from West End Core, Fitzrovia, Soho, Victoria and Marylebone
dev_pipe["New_subm"] = np.where( ((dev_pipe['New_subm'] == "Victoria") | (dev_pipe['New_subm'] == "Soho") | (dev_pipe['New_subm'] == "Fitzrovia") | (dev_pipe['New_subm'] == "Marylebone") | (dev_pipe['New_subm'] == "West End Core")), "West End", dev_pipe["New_subm"] ) 

In [ ]:
# only keep relevant columns
dev_pipe = dev_pipe[['New_subm','Total Development Size','Earliest Possible PC Year']]

In [ ]:
# rename columns - Year and Submarket
dev_pipe=dev_pipe.rename(columns = {'Earliest Possible PC Year':'year', 'New_subm':'Submarket'})

In [ ]:
# sum by submarket and year
dev_pipe = dev_pipe.groupby(['Submarket','year'])['Total Development Size'].sum().reset_index()

In [ ]:
# keep only City and West End 
dev_pipe = dev_pipe[(dev_pipe['Submarket'] == "City") | (dev_pipe['Submarket'] == "West End") ]

In [ ]:
#dev_pipe

## CoStar data

In [ ]:
cstar_city = pd.read_excel('F:\\Research\\CommercialPrivate\\Central London\\DATA\\FORECASTS\\2021\\Q2 2021\\CoStar Stock and Vacancy City - 22 June 2021.xlsx', 'DataExport')
cstar_westend = pd.read_excel('F:\\Research\\CommercialPrivate\\Central London\\DATA\\FORECASTS\\2021\\Q2 2021\\CoStar Stock and Vacancy West End - 22 June 2021.xlsx', 'DataExport')

In [ ]:
cstar_city['Submarket'] = 'City'
cstar_westend['Submarket'] = 'West End'

In [ ]:
# append the datasets
costar = cstar_city.append(cstar_westend, ignore_index=True)
costar['year'] = costar['Period'].str[0:4].astype(int)
costar['stock_growth'] = costar.groupby(['Submarket'])['Inventory SF'].pct_change()
costar = costar[['year', 'Submarket' ,'stock_growth', 'Vacancy Rate']]
#product_df.groupby('prod_desc')['prod_count'].pct_change()

In [ ]:
costar

## Merging the datasets

In [ ]:
# merge PMA Rent and Stock Data and GDP deflator
pma_and_defl  = pd.merge(pma, defl, on='year', how = 'right') # this will create empty rows

In [ ]:
pma_and_defl["Submarket"] = np.where(pma_and_defl['year']>2020,"City",pma_and_defl["Submarket"])  # replace existing empty rows with City

In [ ]:
# create duplicate rows and replace for West End
dupl = pma_and_defl.iloc[82:88].copy()
dupl.columns = pma_and_defl.columns
dupl["Submarket"] = "West End"
#print(dupl)

In [ ]:
# append to the original
pma_and_defl = pma_and_defl.append(dupl, ignore_index=True)
#pma_and_defl

In [ ]:
# add OE data on Employment and GVA
pma_and_defl_and_OE =  pd.merge(pma_and_defl, df_aggr, on='year')

In [ ]:
# add Dev Pipeline
pma_and_defl_and_OE_and_devpipe =  pd.merge(pma_and_defl_and_OE, dev_pipe, on=['year', 'Submarket'], how = 'left')

In [ ]:
#pma_and_defl_and_OE_and_devpipe

## Creating forecast for stock

In [ ]:
final =  pma_and_defl_and_OE_and_devpipe.copy()

In [ ]:
# Add demolitions column based on PMA's advice - 1200 for West End and 1300 for City
final['Demolitions'] = 1200
final['Demolitions'] = np.where(final['Submarket'] =='City',1300,final['Demolitions']) 
final['Demolitions'] = np.where(final['year'] <2021 ,0,final['Demolitions'] ) 

In [ ]:
# divide the Development number by 1000 so that all stock is in the same units
# replace NAs with zeros so that the summing works

final['Total Development Size'] = final['Total Development Size']/1000
final['Total Development Size'] = final['Total Development Size'].fillna(0)

In [ ]:
# create Stock forecast
final = final.sort_values(by=['Submarket', 'year'])

In [ ]:
final['stock'] = final.groupby(['Submarket'])['stock'].fillna(method='ffill') #fill stock values forward
final['Cum_Demol'] = final.groupby(['Submarket'])['Demolitions'].cumsum() # create running total of Demolitions
final['Cum_Devel'] = final.groupby(['Submarket'])['Total Development Size'].cumsum() # create running total of Developments

In [ ]:
# calculate future stock
final['stock'] = np.where(final['year'] >2020,final['stock']+final['Cum_Devel']-final['Cum_Demol'],final['stock'] ) 
#final

In [ ]:
# add CoStar stock growth and vacancy
final =  pd.merge(final, costar, on=['year', 'Submarket'], how = 'left')
#final

In [ ]:
# keep only relevant columns

In [ ]:
final = final.drop(['Cum_Devel', 'Cum_Demol'], axis=1)
final